Mainland RDJ grids (excluding islands) filtered on road network are fed into this data.

## Imports and install

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from shapely.geometry import Polygon,Point
from shapely.ops import unary_union
from tqdm import tqdm
import pandas as pd

In [6]:
# Define output folder path
output_folder_path = os.path.join('..', '..', 'data','02_intermediate_output')

# Define data to process folder path
data_folder_path = os.path.join('..','..', 'data','01_input_data')

In [7]:
# Read safety score grid file
grids_path = os.path.join(data_folder_path, '00_zonal_stats_processed_data.gpkg')
grids = gpd.read_file(grids_path)

In [5]:
# If 'ID' column doesn't exist, create it with unique identifiers
if 'Id' not in grids.columns:
    grids['Id'] = range(1, len(grids) + 1)

## Distance to features

In [6]:
# read both points data.
police_stations = gpd.read_file(os.path.join(data_folder_path, 'policestations.gpkg'))
military_stations = gpd.read_file(os.path.join(data_folder_path, 'military_police_battolions.gpkg'))


In [7]:

def find_nearest_stations(grids, stations, distance_column, geometry_column='geometry'):
    # Calculate centroids of zonal_stats_zones
    grids_centroids = gpd.GeoDataFrame(geometry=grids[geometry_column].centroid)
    
    # Perform spatial join to find nearest station
    joined_data = gpd.sjoin_nearest(grids_centroids, stations, distance_col=distance_column, lsuffix="left", rsuffix="right")
    
    # Add distance to closest station to zonal_stats_zones
    grids[distance_column] = joined_data[distance_column]


In [8]:

# Find nearest police station and military station for each centroid in zonal_stats_zones
find_nearest_stations(grids, police_stations, 'Distance_to_closest_police_station')
find_nearest_stations(grids, military_stations, 'Distance_to_closest_military_station')

## CISP Data

! The crime data from isp has already been processed to have record of crimes per cisp region

In [28]:
cisp_crime_path = os.path.join(output_folder_path, 'crime_counts_per_cisp.csv') # load csv from output folder
crime_counts_per_cisp = pd.read_csv(cisp_crime_path)
crime_counts_per_cisp.head()

,cisp,intentional_homicide_total,bodily_injury_death_total,robbery_with_death_total,violent_crime_total,homicide_by_police_intervention_total,violent_lethality_total,attempted_homicide_total,intentional_bodily_injury_total,rape_total,...,federal_police_arrest_total,apai_arrest_total,arrest_warrant_total,search_warrant_and_seizure_total,threat_total,missing_people_total,cadaver_found_total,skeleton_found_total,military_police_dead_in_service_total,civil_police_dead_in_service_total
0,1,84,3,19,106,7,113,117,4891,141,...,4564.0,1085.0,2818.0,305.0,4672,301,61,3,0,0
1,4,433,19,18,470,101,571,838,7864,345,...,4865.0,1461.0,2220.0,978.0,5014,822,173,4,4,1
2,5,231,7,21,259,29,288,357,12569,462,...,9885.0,2507.0,3999.0,695.0,9663,694,147,1,4,1
3,6,800,15,25,840,379,1219,1180,9812,521,...,3837.0,954.0,2259.0,219.0,6703,708,107,3,14,1
4,7,185,1,8,194,105,299,517,3624,200,...,936.0,276.0,455.0,53.0,3092,242,63,12,5,0


In [26]:
grids = pd.merge(grids, crime_counts_per_cisp, left_on='Cisp_ID', right_on='cisp', how='left') # merge crime counts per cisp with grids
grids.drop(columns=['cisp'], inplace=True)
grids.fillna(0, inplace=True)
grids

,Cumulative_opportunity_measure_to_access_jobs_in_60_minutes,Travel_time_to_closest_healthcare_facility,Number_of_buildings,Illiteracy_rate,Average_household_income,Satellite_nightlight_saturation,Number_of_streetlights,Number_of_CCTV_camera,Average_building_height,Normalized_difference_vegetation_index,...,federal_police_arrest_total,apai_arrest_total,arrest_warrant_total,search_warrant_and_seizure_total,threat_total,missing_people_total,cadaver_found_total,skeleton_found_total,military_police_dead_in_service_total,civil_police_dead_in_service_total
0,0.000000,30.0,0,0.0,0.0,87.756094,0.0,0.0,0.0,0.605964,...,4394.0,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0
1,0.000000,30.0,0,0.0,0.0,88.000000,0.0,0.0,0.0,0.641408,...,4394.0,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0
2,0.000000,30.0,0,0.0,0.0,88.000000,0.0,0.0,0.0,0.634849,...,4394.0,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0
3,0.000000,30.0,0,0.0,0.0,82.989711,0.0,0.0,0.0,0.644144,...,4394.0,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0
4,0.000000,30.0,0,0.0,0.0,85.700592,0.0,0.0,0.0,0.675558,...,4394.0,576.0,3235.0,70.0,29204.0,3124.0,283.0,12.0,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29646,0.000000,30.0,0,0.0,0.0,257.000000,0.0,0.0,0.0,0.100000,...,4123.0,1363.0,1096.0,144.0,6970.0,585.0,175.0,6.0,1.0,1.0
29647,0.000000,30.0,0,0.0,0.0,257.000000,0.0,0.0,0.0,0.123916,...,4123.0,1363.0,1096.0,144.0,6970.0,585.0,175.0,6.0,1.0,1.0
29648,83525.000000,30.0,0,0.0,0.0,94.000000,0.0,0.0,0.0,0.337533,...,4123.0,1363.0,1096.0,144.0,6970.0,585.0,175.0,6.0,1.0,1.0
29649,83332.587021,30.0,0,0.0,0.0,52.587610,0.0,0.0,0.0,0.342686,...,4123.0,1363.0,1096.0,144.0,6970.0,585.0,175.0,6.0,1.0,1.0


## Coverage calculation

In [14]:
# Read urban and favela data
urban_and_favela_gdf_path = os.path.join(data_folder_path,  'landuse.gpkg')
urban_and_favela_gdf = gpd.read_file(urban_and_favela_gdf_path)

# Read building data
building_gdf_path = os.path.join(data_folder_path, 'buildings_footprints.gpkg')
building_gdf = gpd.read_file(building_gdf_path)

# Read gang territories data
gang_gdf_path = os.path.join(data_folder_path,  'gangs_territories.gpkg')
gang_gdf = gpd.read_file(gang_gdf_path)

# Read pacifying police territories data
upp_path = os.path.join(data_folder_path, 'pacifying_police_territories.gpkg')
upp = gpd.read_file(upp_path)

In [15]:
def calculate_percentage(grid_gdf, target_gdf, result_name):
    # Create the spatial index for the target GeoDataFrame
    target_sindex = target_gdf.sindex

    # Iterate through the grid GeoDataFrame
    for index, grid_row in tqdm(grid_gdf.iterrows(), total=len(grid_gdf), desc=f"Processing {result_name}"):
        # Access the geometry of the grid row
        grid_polygon = grid_row.geometry
        
        # Get the bounds of the grid polygon
        grid_bounds = grid_polygon.bounds
        
        # Query the target GeoDataFrame using the spatial index
        possible_matches_index = list(target_sindex.intersection(grid_bounds))
        possible_matches = target_gdf.iloc[possible_matches_index]
        
        # Filter possible matches by intersection with grid polygon
        intersect_poly = possible_matches[possible_matches.intersects(grid_polygon)]
        
        if not intersect_poly.empty:
            intersection_geometry = grid_polygon.intersection(intersect_poly.unary_union)
            clipped_area = intersection_geometry.area

            grid_area = grid_polygon.area
            area_ratio = min((clipped_area / grid_area) * 100, 100) if grid_area != 0 else 0

        else:
            area_ratio = 0
            clipped_area = 0

        # Assign the calculated values directly to the grid_gdf DataFrame
        grid_gdf.at[index, f"{result_name}_Area"] = clipped_area
        grid_gdf.at[index, f"Coverage_by_{result_name}_area"] = area_ratio
    
    return grid_gdf


In [27]:
 # Calculate percentage coverage for urban areas, favelas,  buildings and upp
calculate_percentage(grids, urban_and_favela_gdf[urban_and_favela_gdf['Grupo'] == 'Áreas urbanizadas'], "Urban") # filter urban areas
calculate_percentage(grids, urban_and_favela_gdf[urban_and_favela_gdf['UsoAgregad'] == 'Favela'], 'Favela') # filter favelas
calculate_percentage(grids, building_gdf, 'Building') 
calculate_percentage(grids, upp, 'pacification_police')

In [28]:
# function to Calculate the percentage coverage of different gangs within each grid cell.
    
def calculate_gang_percentage(grid_gdf, gang_gdf):
    # Create spatial index for the polygon GeoDataFrame
    gang_sindex = gang_gdf.sindex

    # Get unique values (group names) in the "fac_nam" column
    unique_fac_names = gang_gdf["fac_nam"].unique()

    # Track added columns to avoid duplicates
    added_columns = []

    # Loop through each grid polygon
    for index, grid_row in tqdm(grid_gdf.iterrows(), total=len(grid_gdf),desc="Processing Grids"):
        grid_polygon = grid_row["geometry"]
        grid_area = grid_polygon.area

        # Use spatial index to find potentially intersecting polygons
        possible_matches_index = list(gang_sindex.intersection(grid_polygon.bounds))
        possible_matches = gang_gdf.iloc[possible_matches_index]

        # Clip polygons to the extent of the current grid polygon
        clipped_polygons = gpd.clip(possible_matches, grid_polygon)

        # Check if clipped_polygons is empty or grid_area is 0
        if clipped_polygons.empty or grid_area == 0:
            continue

        # Calculate the total area of clipped polygons
        clipped_area_total = clipped_polygons.area.sum()

        grid_gdf.at[index, "Percentage"] = min((clipped_area_total / grid_area) * 100, 100)

        # Add columns for each unique value in "fac_nam" with corresponding percentage
        for fac_name in unique_fac_names:
            # Calculate the percentage for each gang
            fac_name_percentage = (clipped_polygons[clipped_polygons["fac_nam"] == fac_name].area.sum() / grid_area) * 100

            # Check if the column for this gang has already been added
            column_name = f"{fac_name}_Percentage"
            if column_name not in added_columns:
                # Add the column if it's not already added
                grid_gdf[column_name] = 0.0
                added_columns.append(column_name)

            # Assign the percentage value
            grid_gdf.at[index, column_name] = fac_name_percentage

    return grid_gdf


In [24]:
# Calculate percentage coverage for gang areas
perc_gdf = calculate_gang_percentage(grids, gang_gdf)


Processing Grids: 100%|██████████| 31248/31248 [07:38<00:00, 68.22it/s] 


In [ ]:
# Rename the columns 
perc_gdf = perc_gdf.rename(columns={
    'Building_Area': 'Coverage_of_built_up_area',
    'Coverage_by_Building_area': 'Building_completeness'
    'Em disputa_Percentage': 'Coverage_by_Em_disputa_gang_area',
    'Milicia_Percentage': 'Coverage_by_Militia_gang_area',
    'Terceiro Comando Puro_Percentage': 'Coverage_by_Terceiro_Comando_Puro_gang_area',
    'Amigo dos amigos_Percentage': 'Coverage_by_Amigo_dos_amigos_gang_area',
    'Comando Vermelho_Percentage': 'Coverage_by_Comando_Vermelho_gang_area',
})

# List of column names to drop
columns_to_drop = ['Favela_Area', 'Building_Area', 'Percentage_Urban_Area']

# Drop unnecessary columns
final_gdf = perc_gdf.drop(columns=columns_to_drop)

## save

In [ ]:
# Save the GeoDataFrame to GeoPackage format with the specified driver
final_gdf.to_file(output_folder_path + '/crime_index_zonal_statistics.gpkg')

# Drop the geometry column before saving to CSV
# final_gdf.drop(columns='geometry', inplace=True)

# # Save the GeoDataFrame to CSV format
# final_gdf.to_csv(output_folder_path + '/crime_index_zonal_statistics.csv', index=False)
